# Project 2 Part 2: API Calls
---

Author: Jackson Muehlbauer

Date: 3/1/2023

In [6]:
# imports 
import pandas as pd
import numpy as np
import json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import os, time

In [7]:
# define function to save movie review with other movie data

def get_movie_with_rating(imdb_id):
    # make API call to tmdb 
    movie = tmdb.Movies(imdb_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    
    return info

In [8]:
# Function to append or extend json file

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [9]:
# Fetching login creds to tmdb
with open('/Users/jacksonmuehlbauer/.secret/tmdb_api.json') as f:
    login = json.load(f)
    
login.keys()

dict_keys(['API Key (v3 auth)', 'API Read Access Token (v4 auth)'])

In [10]:
# set api key
tmdb.API_KEY = login['API Key (v3 auth)']

In [11]:
# Testing get_movie_with_rating function
get_movie_with_rating('tt0848228') # The Avengers

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 150.823,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [12]:
# Testing get_movie_with_rating function
get_movie_with_rating('tt0332280') # The Avengers

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 55.267,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [15]:
# Designate a folder

FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [16]:
# Load in the dataframe from project part 1 as basics
basics = pd.read_csv('Data/title_basics.csv.gz')

In [17]:
# Two desired years
YEARS_TO_GET = [2000, 2001]

In [18]:
# Errors
errors = []

In [19]:
# Outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):
    
    # Define json filename
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if json file exists, if not, make it
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}], f)
    
    # Saving year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    
    # Saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # Get index and movie id from list
    # INNER loop
    for movie_id in tqdm_notebook(movie_ids_to_get, desc = f'Movies from {YEAR}', position = 1, leave = True):
        try:
            #retrieve the data for the movie id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using pre-made function
            write_json(temp, JSON_FILE)
            # short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
    
    # Save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression = 'gzip', index = False)
    
print(f"Total Errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1432 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1548 [00:00<?, ?it/s]

Total Errors: 455
